In [2]:
from urllib.request import urlopen #打开和浏览url中的内容
from bs4 import BeautifulSoup #解析html文件
import pandas as pd
import numpy as np
import re #匹配正则表达式

我们要抓取北京各个区二手房频道，所以前面城市、频道名称不会变，属于固定部分；后面的城区要遍历北京各个区（东城、西城、朝阳…），页码需要在1-所选城区总页数间变化，属于可变部分。将URL分为两部分，前面的固定部分赋值给url，后面的可变部分用双层for循环遍历，外层循环遍历城区，内层循环遍历所选城区的页面。其中，需要获取所选城区包含的总页数，提取div标签中class=page-box fr的第一个子标签属性page-data的值。

In [24]:
chengqu={'dongcheng':'东城区','xicheng':'西城区','chaoyang':'朝阳区','haidian':'海淀区','fengtai':'丰台区','shijingshan':'石景山区',
         'tongzhou':'通州区','changping':'昌平区','daxing':'大兴区','shunyi':'顺义区','fangshan':'房山区'}

for cq in chengqu.keys():
    url='https://bj.lianjia.com/ershoufang/'+cq+'/'   #组成所选城区的URL
    html=urlopen(url)
    bsObj=BeautifulSoup(html)
    total_page=re.sub('\D','',bsObj.find('div','page-box fr').contents[0].attrs['page-data'])[:-1]  #获取所选城区总页数
    #print ('total_page',total_page)
    
    for j in np.arange(1,3): #我们只爬取前十页的内容
        page_url=url+'pg'+str(j)  #组成所选城区页面的URL
        print (page_url)
        page_html=urlopen(page_url)
        page_bsObj=BeautifulSoup(page_html)
        info=page_bsObj.findAll("div",{"class":"houseInfo"})
        position_info=page_bsObj.findAll("div",{"class":"positionInfo"})
        totalprice=page_bsObj.findAll("div",{"class":"totalPrice"})
        unitprice=page_bsObj.findAll("div",{"class":"unitPrice"})
                
        house_loc=[]  #房屋所在小区
        house_type=[] #房屋户型
        house_area=[] #房屋面积
        house_direction=[] #房屋朝向
        house_decorating=[] #房屋装修
        house_lift=[] #有无电梯
        house_floor=[] #房屋楼层
        house_year=[] #建造年份
        house_position=[] #房屋位置
        t_price=[] #房屋总价
        u_price=[] #房屋单价

        for i_info,i_pinfo,i_tp,i_up in zip(info,position_info,totalprice,unitprice):
            #分列houseinfo并依次获取房屋所在小区、户型、面积、朝向、装修、有无电梯各字段
            house_loc.append(i_info.get_text().split('/')[0])
            house_type.append(i_info.get_text().split('/')[1])
            house_area.append(i_info.get_text().split('/')[2][:-2])
            house_direction.append(i_info.get_text().split('/')[3].replace(' ',''))
            house_decorating.append(i_info.get_text().split('/')[4])
            #house_lift.append(i_info.get_text().split('/')[5])
            #分列positioninfo并依次获房屋楼层、建造年份、位置各字段
            house_floor.append(i_pinfo.get_text().split('/')[0])
            house_year.append(i_pinfo.get_text().split('/')[1][:5])
            house_position.append(i_pinfo.get_text().split('/')[2])
            #获取房屋总价和单价
            t_price.append(i_tp.span.string)
            u_price.append(re.sub('\D','',i_up.get_text()))
        #将数据导入pandas之中生成数据表
        house_data=pd.DataFrame()
        house_data[u'城区']=[chengqu[cq]]*len(house_loc)
        house_data[u'小区名称']=house_loc
        house_data[u'房型']=house_type
        house_data[u'面积']=house_area
        house_data[u'朝向']=house_direction
        house_data[u'装修']=house_decorating
        #house_data[u'有无电梯']=house_lift
        house_data[u'楼层']=house_floor
        house_data[u'建造年份']=house_year
        house_data[u'位置']=house_position
        house_data[u'总价']=t_price
        house_data[u'单价']=u_price
        print(house_data)
        #将数据存入到csv中，便于后续分析
        house_data.to_csv('house_bj.csv', mode='a', header=False)


D:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file D:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


https://bj.lianjia.com/ershoufang/dongcheng/pg1
     城区        小区名称    房型      面积  朝向  装修         楼层   建造年份    位置    总价  \
0   东城区      前门东大街   2室1厅    59.5  东北  精装  中楼层(共16层)  1979年   崇文门   528   
1   东城区       华龙美晟   2室1厅   80.23   东  精装  高楼层(共12层)  2009年   永定门   615   
2   东城区       柏林寺西   2室1厅   58.45   南  简装  中楼层(共16层)  1994年   和平里   640   
3   东城区        南水关   2室1厅   62.05  南北  简装  中楼层(共14层)  1989年   广渠门   570   
4   东城区       华龙美晟   2室1厅   81.38  西北  简装  中楼层(共12层)  2009年   永定门   610   
5   东城区     新景家园东区   1室1厅   56.16   南  简装  低楼层(共13层)  2006年   崇文门   710   
6   东城区       安化北里   2室1厅   69.66   南  精装  低楼层(共17层)  1989年   广渠门   580   
7   东城区    东花市北里中区   3室1厅   72.74  南北  简装   高楼层(共6层)  1994年   东花市   736   
8   东城区       北京上舍   3室1厅  133.06  西南  精装  高楼层(共15层)  2008年   广渠门  1330   
9   东城区      金鱼池西区   1室0厅   45.97   东  精装   地下室(共7层)  2000年    天坛   288   
10  东城区     本家润园三期   2室1厅   77.71  南北  简装  低楼层(共18层)  2005年   东花市   700   
11  东城区    永康胡同1号院   3室1厅   85.07  东西  简装   中楼层(共7层)

     城区        小区名称    房型      面积   朝向  装修         楼层   建造年份    位置    总价  \
0   西城区       教子胡同   2室1厅   57.89   南北  精装   中楼层(共6层)  1980年    牛街   525   
1   西城区      平原里小区   2室1厅   63.19   南北  精装    底层(共6层)  1998年    牛街   635   
2   西城区        北露园   1室1厅    40.3    东  简装   高楼层(共6层)  1993年   阜成门   520   
3   西城区       槐柏树街   2室1厅   58.59   东南  简装  高楼层(共15层)  1986年   长椿街   650   
4   西城区      常青藤嘉园   1室1厅   62.71    南  精装  中楼层(共16层)  2004年   马连道   570   
5   西城区       裕中西里   3室1厅    78.7   东南  精装  高楼层(共20层)  1989年    马甸  1023   
6   西城区        二龙路   2室1厅    65.3   西北  精装  低楼层(共17层)  1990年   金融街  1043   
7   西城区      三里河一区   2室1厅    57.8   南北  简装   中楼层(共6层)  1989年    月坛   800   
8   西城区   前半壁街35号院   1室1厅    39.4    南  简装  中楼层(共13层)  1986年    官园   550   
9   西城区    太仆寺33号院   3室1厅      82   东西  简装   低楼层(共7层)  2001年    西单  1165   
10  西城区     白广路6号院   3室1厅   85.11   南北  毛坯   高楼层(共6层)  1994年    牛街   760   
11  西城区       裕中东里   3室1厅    77.8  东南北  简装   中楼层(共6层)  1986年    马甸  1049   
12  西城区     

     城区       小区名称    房型      面积   朝向  装修         楼层   建造年份      位置    总价  \
0   海淀区   三里河路5号院   1室1厅   44.42    东  简装  低楼层(共18层)  1990年     甘家口   440   
1   海淀区      牛奶宿舍   2室1厅    62.6   南北  精装   中楼层(共6层)  1991年     牡丹园   600   
2   海淀区    富力桃园B区   2室1厅   94.36   东西  精装  中楼层(共11层)  2007年     西三旗   579   
3   海淀区        菊园   2室1厅   64.35   南北  精装         6层  1995年     马连洼   475   
4   海淀区    富力桃园C区   2室1厅   89.58   南北  精装  高楼层(共18层)  2008年     西三旗   560   
5   海淀区       展春园   2室1厅    57.6   南北  简装   中楼层(共6层)  1984年     五道口   485   
6   海淀区    万寿路1号院   3室1厅   87.89   南北  精装    底层(共6层)  1996年     万寿路   815   
7   海淀区      乐府江南   3室2厅  135.44   南北  简装   中楼层(共9层)  2003年      田村  1200   
8   海淀区     小南庄社区   1室1厅   54.31   南北  简装   中楼层(共6层)  1995年     苏州桥   530   
9   海淀区      冶金大院   2室1厅    54.1   南北  精装   中楼层(共5层)  1986年     皂君庙   480   
10  海淀区  学院南路66号院   2室1厅    65.6    南  简装  中楼层(共14层)  1987年     白石桥   570   
11  海淀区       城华园   2室1厅   64.95   南北  精装   高楼层(共6层)  1998年     学院路   565   

      城区      小区名称    房型      面积  朝向  装修         楼层   建造年份   位置   总价     单价
0   石景山区     老山东里   3室1厅      65  西北  简装  低楼层(共16层)  1986年   老山  328  50462
1   石景山区    模式口西里   2室1厅    55.6   南  简装    顶层(共6层)  1987年  苹果园  239  42986
2   石景山区    远洋沁山水   2室1厅   88.74   西  精装  低楼层(共20层)  2009年   鲁谷  700  78883
3   石景山区     远洋山水   2室2厅  109.57  南北  简装  中楼层(共26层)  2007年   鲁谷  675  61605
4   石景山区     古城南里   1室1厅   45.05   南  毛坯  中楼层(共16层)  1984年   古城  260  57714
5   石景山区     杨庄小区   2室1厅      64  南北  精装  低楼层(共14层)  1992年   杨庄  316  49375
6   石景山区     永乐东区   1室1厅   29.07   南  简装   中楼层(共4层)  1958年   鲁谷  160  55040
7   石景山区     远洋山水   2室1厅   96.26  南北  精装   底层(共27层)  2007年   鲁谷  580  60254
8   石景山区   地铁古城家园   2室1厅   79.84  东南  简装  低楼层(共24层)  2000年   杨庄  349  43713
9   石景山区     老山东里   3室1厅      65  西北  简装  高楼层(共16层)  1985年   老山  320  49231
10  石景山区    古城路小区   2室1厅   48.86  东西  精装   中楼层(共6层)  1980年   古城  228  46664
11  石景山区    八角路小区   2室0厅   37.07  南北  简装    顶层(共5层)  1971年   八角  155  41813
12  石景山区    

     城区     小区名称    房型      面积   朝向  装修         楼层   建造年份    位置   总价     单价
0   昌平区   华龙苑北里   2室2厅   90.92   南北  精装   低楼层(共7层)  2005年    霍营  448  49275
1   昌平区    太平家园   2室1厅      71   南北  精装   中楼层(共6层)  1998年   天通苑  380  53522
2   昌平区     龙回苑   3室1厅   116.5   南北  简装   高楼层(共6层)  2004年   回龙观  479  41116
3   昌平区     新龙城   2室1厅  101.93   南北  精装  低楼层(共20层)  2007年   回龙观  499  48956
4   昌平区  天通苑本五区   3室1厅  160.07   南北  精装  低楼层(共13层)  2001年   天通苑  609  38046
5   昌平区     龙华园   2室1厅   80.16   南北  精装    顶层(共6层)  1996年   回龙观  386  48154
6   昌平区   龙腾苑三区   3室1厅  117.31   南北  精装   中楼层(共5层)  2002年   回龙观  518  44157
7   昌平区     西环里   2室1厅   50.16    南  精装   中楼层(共5层)  1984年  西关环岛  215  42863
8   昌平区    旗胜家园   2室1厅    89.4    南  精装   底层(共16层)  2009年   西三旗  405  45303
9   昌平区  天通苑北二区   2室2厅  126.06   南北  精装  中楼层(共13层)  2005年   天通苑  515  40854
10  昌平区    融泽嘉园   2室1厅   92.18   南北  简装  低楼层(共18层)  2014年   回龙观  510  55327
11  昌平区  领秀慧谷D区   2室1厅   80.68   南北  精装  高楼层(共11层)  2016年   回龙观  480  59495
12  昌平区  天通苑

     城区        小区名称    房型      面积  朝向  装修         楼层   建造年份    位置   总价     单价
0   顺义区       仓上小区   1室1厅    60.2  南北  简装    顶层(共6层)  1992年   顺义城  235  39037
1   顺义区       滨河小区   1室1厅    60.9  南北  精装   中楼层(共5层)  1992年   顺义城  235  38588
2   顺义区       石园北区   2室1厅   78.31  南北  精装  中楼层(共16层)  2002年   顺义城  282  36011
3   顺义区       佳和宜园   2室1厅   89.36  南北  简装    顶层(共6层)  2009年    马坡  275  30775
4   顺义区       新马家园   2室1厅   80.93  南北  简装  低楼层(共28层)  2013年    马坡  279  34475
5   顺义区       香悦四季   3室1厅  119.85  南北  精装  低楼层(共16层)  2013年    马坡  495  41302
6   顺义区     金汉绿港四区   3室1厅  106.13  东南  精装   顶层(共20层)  2011年   顺义城  468  44097
7   顺义区    前进花园玉兰苑   2室1厅   96.79  南北  精装  中楼层(共11层)  2007年   顺义城  389  40191
8   顺义区       裕龙五区   3室1厅  146.21  南北  精装   中楼层(共5层)  2003年   顺义城  625  42747
9   顺义区     空港吉祥花园   3室2厅   98.19  南北  精装  低楼层(共11层)  2007年   后沙峪  486  49496
10  顺义区     港馨家园东区   2室1厅   85.09  南北  简装   低楼层(共6层)  2007年   顺义城  285  33494
11  顺义区       西辛南区   2室1厅   59.24   南  精装   中楼层(共6层)  1996年   顺义

In [17]:
data=pd.DataFrame()
data['area']=['就打算']
data=pd.DataFrame()
data

""


依次把页面div标签中class为houseInfo、positionInfo、totalPrice、unitPrice的部分提取出来，用get_text()获取对应标签中的信息，其中houseInfo、positionInfo包含房屋的多种属性，用split()对其进行分列，只选取了houseInfo包含6个属性且positionInfo包含3个属性的数据。另外，对于单价用get_text()后得到“单价100748元/平米”，包含了文本和数字，用re.sub()正则匹配出数字部分。最后，依次把提取到的信息添加到对应列表中。

In [6]:
info=bsObj.findAll("div",{"class":"houseInfo"})
position_info=bsObj.findAll("div",{"class":"positionInfo"})
totalprice=bsObj.findAll("div",{"class":"totalPrice"})
unitprice=bsObj.findAll("div",{"class":"unitPrice"})
house_loc=[]  #房屋所在小区
house_type=[] #房屋户型
house_area=[] #房屋面积
house_direction=[] #房屋朝向
house_decorating=[] #房屋装修
house_lift=[] #有无电梯
house_floor=[] #房屋楼层
house_year=[] #建造年份
house_position=[] #房屋位置
t_price=[] #房屋总价
u_price=[] #房屋单价
for i_info,i_pinfo,i_tp,i_up in zip(info,position_info,totalprice,unitprice):
     if len(i_info.get_text().split('/'))==6 and len(i_pinfo.get_text().split('/'))==3:
         #分列houseinfo并依次获取房屋所在小区、户型、面积、朝向、装修、有无电梯各字段
        house_loc.append(i_info.get_text().split('/')[0])
        house_type.append(i_info.get_text().split('/')[1])
        house_area.append(i_info.get_text().split('/')[2][:-2])
        house_direction.append(i_info.get_text().split('/')[3].replace(' ',''))
        house_decorating.append(i_info.get_text().split('/')[4])
        house_lift.append(i_info.get_text().split('/')[5])
        #分列positioninfo并依次获房屋楼层、建造年份、位置各字段
        house_floor.append(i_pinfo.get_text().split('/')[0])
        house_year.append(i_pinfo.get_text().split('/')[1][:5])
        house_position.append(i_pinfo.get_text().split('/')[2])
        #获取房屋总价和单价
        t_price.append(i_tp.span.string)
        u_price.append(re.sub('\D','',i_up.get_text()))

In [7]:
#将数据导入pandas之中生成数据表
house_data=pd.DataFrame()
house_data[u'城区']=[chengqu[cq]]*len(house_loc)
house_data[u'小区名称']=house_loc
house_data[u'房型']=house_type
house_data[u'面积']=house_area
house_data[u'朝向']=house_direction
house_data[u'装修']=house_decorating
house_data[u'有无电梯']=house_lift
house_data[u'楼层']=house_floor
house_data[u'建造年份']=house_year
house_data[u'位置']=house_position
house_data[u'总价']=t_price
house_data[u'单价']=u_price
print (house_data)


     城区         小区名称    房型      面积   朝向  装修 有无电梯         楼层   建造年份  位置   总价  \
0   房山区     首开熙悦山澜庭   2室1厅   69.74    南  简装  有电梯  高楼层(共21层)  2016年  长阳  310   
1   房山区      金地朗悦朗园   3室1厅   89.31   南北  简装  有电梯   中楼层(共8层)  2011年  良乡  410   
2   房山区     长阳半岛4号院   3室1厅   95.92   南北  精装  有电梯   高楼层(共9层)  2013年  长阳  470   
3   房山区      原香小镇二区   2室1厅   88.77   南北  简装  有电梯    底层(共6层)  2010年  长阳  445   
4   房山区      加州水郡三期   1室1厅   56.47   东北  精装  有电梯  中楼层(共15层)  2009年  长阳  234   
5   房山区       鸿顺园西区   2室1厅   89.79   南北  简装  无电梯   中楼层(共6层)  2003年  良乡  291   
6   房山区       北潞春家园   2室1厅   89.73   东南  简装  无电梯   低楼层(共6层)  2000年  良乡  251   
7   房山区  首开熙悦山熹园1号院   2室1厅   83.13    南  精装  有电梯  中楼层(共21层)  2013年  长阳  346   
8   房山区      加州水郡三期   2室1厅   84.57   南北  精装  有电梯  高楼层(共15层)  2009年  长阳  258   
9   房山区     长阳半岛2号院   2室1厅   87.65   南北  精装  有电梯  中楼层(共28层)  2011年  长阳  370   
10  房山区       西潞园二里   2室1厅   59.35  南西北  简装  无电梯   高楼层(共7层)  1994年  良乡  205   
11  房山区       首创新悦都   2室1厅   75.49   南北  精装  有电梯  中楼

In [49]:
house_data.to_csv('house_bj.csv',mode='a', header=True)

In [50]:
data = pd.read_csv('house_bj.csv')
data.index

RangeIndex(start=0, stop=28, step=1)

In [51]:
data.head()

,Unnamed: 0,城区,小区名称,房型,面积,朝向,装修,有无电梯,楼层,建造年份,位置,总价,单价
0,0,房山区,首开熙悦山澜庭,2室1厅,69.74,南,简装,有电梯,高楼层(共21层),2016年,长阳,310,44451
1,1,房山区,金地朗悦朗园,3室1厅,89.31,南北,简装,有电梯,中楼层(共8层),2011年,良乡,410,45908
2,2,房山区,长阳半岛4号院,3室1厅,95.92,南北,精装,有电梯,高楼层(共9层),2013年,长阳,470,49000
3,3,房山区,原香小镇二区,2室1厅,88.77,南北,简装,有电梯,底层(共6层),2010年,长阳,445,50130
4,4,房山区,加州水郡三期,1室1厅,56.47,东北,精装,有电梯,中楼层(共15层),2009年,长阳,234,41438


0                   2
房山区               房山区
首开熙悦山澜庭      长阳半岛4号院 
2室1厅             3室1厅
69.74           95.92
南                  南北
简装                 精装
有电梯               有电梯
高楼层(共21层)    高楼层(共9层)
2016年           2013年
长阳                 长阳
310               470
44451           49000
Name: 1, dtype: object